In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.test_suite import TestSuite

from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_preset import DataQualityTestPreset
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import DataDriftTestPreset
from evidently.test_preset import RegressionTestPreset
from evidently.test_preset import MulticlassClassificationTestPreset
from evidently.test_preset import BinaryClassificationTopKTestPreset
from evidently.test_preset import BinaryClassificationTestPreset

In [2]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
#Dataset for Regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [4]:
#Dataset for Binary Probabilistic Classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [5]:
#Dataset for Multiclass Classifcation
iris_data = datasets.load_iris(as_frame='auto')
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])

In [ ]:
data_stability = TestSuite(tests=[
    DataStabilityTestPreset(),
])

data_stability.run(reference_data=adult_ref, current_data=adult_cur)
data_stability

In [7]:
#test preset in a JSON format
data_stability.json()

'{"version": "0.4.1", "tests": [{"name": "Number of Rows", "description": "The number of rows is 34687. The test threshold is eq=1.42e+04 \\u00b1 1.42e+03.", "status": "FAIL", "group": "data_integrity", "parameters": {"condition": {"eq": {"value": 14155, "relative": 0.1, "absolute": 1e-12}}, "value": 34687.0}}, {"name": "Number of Columns", "description": "The number of columns is 15. The test threshold is eq=15.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"eq": 15}, "value": 15.0}}, {"name": "Column Types", "description": "The number of columns with a type mismatch is 0 out of 15.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"columns": [{"actual_type": "float64", "column_name": "age", "expected_type": "float64"}, {"actual_type": "CategoricalDtypeType", "column_name": "workclass", "expected_type": "CategoricalDtypeType"}, {"actual_type": "float64", "column_name": "fnlwgt", "expected_type": "float64"}, {"actual_type": "CategoricalDt

In [ ]:
#test preset as a python object
data_stability.as_dict()

In [ ]:
data_quality = TestSuite(tests=[
    DataQualityTestPreset(),
])

data_quality.run(reference_data=adult_ref, current_data=adult_cur)
data_quality

In [ ]:
data_drift = TestSuite(tests=[
    DataDriftTestPreset(stattest='psi'),
])

data_drift.run(reference_data=adult_ref, current_data=adult_cur)
data_drift

In [ ]:
no_target_performance = TestSuite(tests=[
    NoTargetPerformanceTestPreset(columns=['education-num', 'hours-per-week'],  num_stattest='ks', cat_stattest='psi'),
])

no_target_performance.run(reference_data=adult_ref, current_data=adult_cur)
no_target_performance

In [ ]:
regression_performance = TestSuite(tests=[
    RegressionTestPreset()
])

regression_performance.run(reference_data=housing_ref, current_data=housing_cur)
regression_performance

In [ ]:
classification_performance = TestSuite(tests=[
   MulticlassClassificationTestPreset(stattest='psi')
])

classification_performance.run(reference_data=iris_ref, current_data=iris_cur)
classification_performance

In [ ]:
label_binary_classification_performance = TestSuite(tests=[
    BinaryClassificationTestPreset(),
])

label_binary_classification_performance.run(reference_data=bcancer_label_ref, current_data=bcancer_label_cur)
label_binary_classification_performance

In [ ]:
prob_binary_classification_performance = TestSuite(tests=[
    BinaryClassificationTestPreset(stattest='psi', probas_threshold=0.89),
])

prob_binary_classification_performance.run(reference_data=bcancer_ref, current_data=bcancer_cur)
prob_binary_classification_performance